In [14]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [15]:
# Create network model
model = Network('model')

# Block 1: Create nodes
node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.GPSPRIO)  # Generalized Processor Sharing with Priority

print("Network created with nodes:")
print("  Delay: Think time/delay station")
print("  Queue1: GPSPRIO (Generalized Processor Sharing with Priority) queue")

Network created with nodes:
  Delay: Think time/delay station
  Queue1: GPSPRIO (Generalized Processor Sharing with Priority) queue


In [16]:
# Block 2: Create job classes with different priorities and populations
jobclass = np.empty(4, dtype=object)
jobclass[0] = ClosedClass(model, 'Class1', 6, node[0], 0)  # 6 jobs, priority 0
jobclass[1] = ClosedClass(model, 'Class2', 4, node[0], 1)  # 4 jobs, priority 1  
jobclass[2] = ClosedClass(model, 'Class3', 4, node[0], 1)  # 4 jobs, priority 1
jobclass[3] = ClosedClass(model, 'Class4', 1, node[0], 2)  # 1 job, priority 2 (highest)

print("Job classes created:")
print("  Class1: 6 jobs, priority 0 (lowest)")
print("  Class2: 4 jobs, priority 1 (medium)")
print("  Class3: 4 jobs, priority 1 (medium)")
print("  Class4: 1 job,  priority 2 (highest)")
print(f"  Total population: {6+4+4+1} jobs")

Job classes created:
  Class1: 6 jobs, priority 0 (lowest)
  Class2: 4 jobs, priority 1 (medium)
  Class3: 4 jobs, priority 1 (medium)
  Class4: 1 job,  priority 2 (highest)
  Total population: 15 jobs


In [17]:
# Block 3: Set service time distributions at Delay station
node[0].set_service(jobclass[0], Erlang(3, 2))  # Erlang(k=3, rate=2)
node[0].set_service(jobclass[1], Exp.fitMean(1.0))  # Exponential with mean=1.0
node[0].set_service(jobclass[2], Exp.fitMean(1.0))  # Exponential with mean=1.0
node[0].set_service(jobclass[3], Exp.fitMean(0.5))  # Exponential with mean=0.5

print("Delay station service times configured:")
print("  Class1: Erlang(k=3, rate=2)")
print("  Class2: Exponential(mean=1.0)")
print("  Class3: Exponential(mean=1.0)")
print("  Class4: Exponential(mean=0.5)")

Delay station service times configured:
  Class1: Erlang(k=3, rate=2)
  Class2: Exponential(mean=1.0)
  Class3: Exponential(mean=1.0)
  Class4: Exponential(mean=0.5)


In [18]:
# Block 4: Set service times and weights at GPSPRIO queue
# Service rates and weights for weighted sharing within priority levels
w1 = 12  # Weight for Class1
w2 = 3   # Weight for Class2
w3 = 5   # Weight for Class3
w4 = 1   # Weight for Class4

# Set service times with weights (Python API: setService(jobclass, distribution, weight))
node[1].set_service(jobclass[0], Exp.fitRate(30.0), w1)  # Rate=30, Weight=12
node[1].set_service(jobclass[1], Exp.fitRate(2.0), w2)   # Rate=2, Weight=3
node[1].set_service(jobclass[2], Exp.fitRate(12.0), w3)  # Rate=12, Weight=5
node[1].set_service(jobclass[3], Exp.fitRate(1.0), w4)   # Rate=1, Weight=1

print("GPSPRIO queue service configuration:")
print(f"  Class1 (priority 0): rate=30.0, weight={w1}")
print(f"  Class2 (priority 1): rate=2.0,  weight={w2}")
print(f"  Class3 (priority 1): rate=12.0, weight={w3}")
print(f"  Class4 (priority 2): rate=1.0,  weight={w4}")
print()
print("GPSPRIO behavior:")
print("  1. Class4 (priority 2) gets served first when present")
print("  2. Class2&3 (priority 1) share capacity by weights 3:5 when present")
print("  3. Class1 (priority 0) gets remaining capacity")

GPSPRIO queue service configuration:
  Class1 (priority 0): rate=30.0, weight=12
  Class2 (priority 1): rate=2.0,  weight=3
  Class3 (priority 1): rate=12.0, weight=5
  Class4 (priority 2): rate=1.0,  weight=1

GPSPRIO behavior:
  1. Class4 (priority 2) gets served first when present
  2. Class2&3 (priority 1) share capacity by weights 3:5 when present
  3. Class1 (priority 0) gets remaining capacity


In [19]:
# Block 5: Set routing (serial routing for all classes)
P = model.init_routing_matrix()

# All classes follow the same cyclic route: Delay <-> Queue1
for i in range(4):
    P.add_route(jobclass[i], node[0], node[1], 1.0)  # Delay -> Queue1
    P.add_route(jobclass[i], node[1], node[0], 1.0)  # Queue1 -> Delay

model.link(P)

print("Routing configured: Delay <-> Queue1 (cyclic for all classes)")
print("All classes follow the same route but with different priorities and weights")

Routing configured: Delay <-> Queue1 (cyclic for all classes)
All classes follow the same route but with different priorities and weights


In [20]:
# Model information
M = model.get_number_of_stations()
K = model.get_number_of_classes()
print(f"Model has {M} stations and {K} classes")
print(f"Total population: {sum(cls.getNumberOfJobs() for cls in jobclass)} jobs")

Model has 2 stations and 4 classes
Total population: 15.0 jobs


In [ ]:
# SolverCTMC with verbose=SILENT to match JAR test  
solver_ctmc = SolverCTMC(model, verbose=VerboseLevel.SILENT)
avg_table_ctmc = solver_ctmc.get_avg_table()
print(avg_table_ctmc)

In [ ]:
# SolverJMT removed - not tested in JAR for prio_identical
# solver_jmt = SolverJMT(model, seed=23000, verbose=VerboseLevel.STD, samples=30000)
# avg_table_jmt = solver_jmt.get_avg_table()
# print(avg_table_jmt)